In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.losses import categorical_crossentropy
from keras.optimizers import adam, sgd
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
def cnn_model():
    model=Sequential()
    model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(50,50,3)))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(500,activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(8,activation="softmax"))
    model.summary()
    return model

In [7]:
train_path = './TERRAIN_DATASET_SPLIT/train'
test_path = './TERRAIN_DATASET_SPLIT/val'
IMG_W = 50
IMG_H = 50
num_classes = 2

# Loads the TRAIN images
train_batch = ImageDataGenerator(featurewise_center=False,
                                 samplewise_center=False, 
                                 featurewise_std_normalization=False, 
                                 samplewise_std_normalization=False, 
                                 zca_whitening=False, 
                                 rotation_range=45, 
                                 width_shift_range=0.2, 
                                 height_shift_range=0.2, 
                                 horizontal_flip=True, 
                                 vertical_flip=False).flow_from_directory(train_path, 
                                                                          target_size=(IMG_H, IMG_W), 
                                                                          classes=['curb', 'footpath', 'grass', 'stones', 'road', 'sand', 'snow', 'wet'], 
                                                                          batch_size=100)

#Loads the TEST images
test_batch = ImageDataGenerator().flow_from_directory(test_path, 
                                                      target_size=(IMG_H, IMG_W), 
                                                      classes=['curb', 'footpath', 'grass', 'stones', 'road', 'sand', 'snow', 'wet'], 
                                                      batch_size=100)

Found 4014 images belonging to 8 classes.
Found 2753 images belonging to 8 classes.


In [4]:
# Create a new model
model = cnn_model()

# If the model had to stop training we can pick up from where we left off
checkpoint = ModelCheckpoint('terrain_model.h5', 
                             monitor='val_loss', 
                             verbose=0, 
                             save_best_only=True, 
                             mode='auto')

# Set the model up for training
model.compile(loss='categorical_crossentropy', optimizer=adam(lr=1.0e-4), metrics=['accuracy'])

# Train the model
history = model.fit_generator(train_batch, 
                             validation_data=test_batch,  
                             epochs=100, 
                             verbose=1, 
                             callbacks=[checkpoint])






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 50, 50, 16)        208       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 25, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 25, 25, 32)        2080      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 64)        8256      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 64)          0         
______________________

ValueError: `steps_per_epoch=None` is only valid for a generator based on the `keras.utils.Sequence` class. Please specify `steps_per_epoch` or use the `keras.utils.Sequence` class.

In [ ]:
model = load_model('terrain_model.h5')
terrain_classes=['curb', 'footpath', 'grass', 'stones', 'road', 'sand', 'snow', 'wet'], 

def classify_image(path):
    pic = plt.imread(path)
    pic = cv2.resize(pic, (IMG_W, IMG_H))
    pic = np.expand_dims(pic, axis=0)
    class_predicted = model.predict_classes(pic)
    print(class_predicted.max())

In [ ]:
classify_image("./stones.jpg")

In [ ]:
sns.set_style("darkgrid")
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()